In [ ]:
%matplotlib inline

import os

import mne
import numpy as np

## Analysis of event-related potentials (ERPs) - the `Evoked` class

ERPs/[evoked](https://mne.tools/stable/documentation/glossary.html#term-evoked) data are another staple of many signal analysis projects.

As the name implies, ERPs are changes in voltage associated with a particular event, e.g. the presentation of a stimulus, or the execution of some action.

The first step in generating ERPs is to epoch the data around the event of interest, after which we generally average across the epochs to generate the final ERPs.

### Part 1 - Creating `Evoked` objects from `Epochs`

We start by loading the sample data, and choosing the stimulus channel from which we want to generate event markers.

We additionally create a dictionary of event labels and their corresponding IDs which we want to generate epochs around.

In [ ]:
# Load the sample data
raw = mne.io.read_raw_fif(
    os.path.join(mne.datasets.sample.data_path(), "MEG", "sample", "sample_audvis_raw.fif")
)
raw.pick(picks=["eeg", "stim"])
raw.del_proj()

# Generate the events array
events = mne.find_events(raw, stim_channel="STI 014")

# Choose the events to create epochs around
event_id = {
    "auditory/left": 1,
    "auditory/right": 2,
    "visual/left": 3,
    "visual/right": 4,
}

**Exercises - Creating `Evoked` objects from `Epochs`**

**Exercise:** Create an [`Epochs`](https://mne.tools/stable/generated/mne.Epochs.html) object from the data called `epochs`.

Pass the `events` and `event_id` variables to the corresponding parameters to specify the events to epoch around.

Using the `tmin` and `tmax` parameters, create epochs around the events in the window [-0.25, 0.75].

*Hint:* Refer to the documentation for the `Epochs` class and the [Epochs notebook](../Day%201/2%20-%20Epochs.ipynb) for a reminder how to do this.

In [ ]:
## CODE GOES HERE

Below, you can see that the numeric IDs of the events in the `Epochs` object have been assigned to more descriptive names provided in the `event_id` dictionary.

In [ ]:
# You should call the `Epochs` object created above "epochs"
epochs.load_data()
epochs

Averaging across the epochs to create ERPs is as simple as calling the [`average()`](https://mne.tools/stable/generated/mne.Epochs.html#mne.Epochs.average) method of the `Epochs` object.

This returns an [`Evoked`](https://mne.tools/stable/generated/mne.Evoked.html#mne.Evoked) object.

An example is shown below.

In [ ]:
# Create the evoked responses for the left auditory stimulus
evoked_aud_l = epochs["auditory/left"].average()
evoked_aud_l

Here, we selected only those events corresponding to the `"auditory/left"` label to average across.

We can visualise the ERPs using the [`plot()`](https://mne.tools/stable/generated/mne.Evoked.html#mne.Evoked.plot) method.

In [ ]:
# Visualise the evoked response
evoked_aud_l.plot();

**Exercise:** Create ERPs for the "auditory/right" stimulus and visualise them.

In [ ]:
## CODE GOES HERE

**Exercise:** Create ERPs for the "visual/left" stimulus and visualise them.

In [ ]:
## CODE GOES HERE

#### Creating ERPs: mean vs. median

By default, calling the `average()` method on `Epochs` objects will generated `Evoked` objects using the mean of the values across the epochs.

However, we can also create `Evoked` objects using the median, or even custom functions.

Below, we create ERPs for the "visual/right" stimulus, explicitly specifying the mean to be used with the `method` parameter.

In [ ]:
# Create evoked responses specifying the mean method explicitly
epochs.load_data()
evoked_vis_r_mean = epochs["visual/right"].average(method="mean")
evoked_vis_r_mean.plot()
evoked_vis_r_mean

**Exercise:** Create ERPs for the "visual/right" stimulus using the median method, and visualise them.

In [ ]:
## CODE GOES HERE

Custom functions can also be used to control how the information is combined across epochs.

For example, the function below uses the data of only every other epoch and takes the mean of this.

In [ ]:
def every_other_epoch_mean(epoched_data: np.ndarray) -> np.ndarray:
    """Take the mean across every other epoch.

    Parameters
    ----------
    epoched_data : numpy.ndarray, shape of (epochs, channels, times)
    - The epochs to create evoked data from.

    Returns
    -------
    evoked_data : numpy.ndarray, shape of (channels, times)
    - The evoked data as the mean across every other epoch.
    """
    # Select every other epoch
    every_other_epoch = epoched_data[::2, :, :]  # [::2] takes every other element

    # Average across the remaining epochs
    evoked_data = np.mean(every_other_epoch, axis=0)

    return evoked_data

We then simply pass this to the `method` parameter of the `average()` method.

In [ ]:
# Create evoked responses using the custom method
evoked_vis_r_custom = epochs["visual/right"].average(method=every_other_epoch_mean)
evoked_vis_r_custom.plot()
evoked_vis_r_custom

#### Handling multiple event types simultaneously

Multiple events types can also be selected at once for processing into `Evoked` objects.

By default, averaging is performed across all selected event types, such that a single `Evoked` object is returned.

In [ ]:
# Create evoked responses for the auditory stimuli
evoked_aud = epochs[["auditory/left", "auditory/right"]].average()
evoked_aud.plot()
evoked_aud

However, you can also specify to only average across events with the same label.

This behaviour is controlled with the `by_event_type` parameter of the `average()` method.

`by_event_type` is by default `False`, which combines the epochs regardless of type.

In [ ]:
# Same behaviour as above (i.e. average across events regardless of type)
evoked_vis = epochs[["visual/left", "visual/right"]].average(by_event_type=False)
evoked_vis.plot()
evoked_vis

On the other hand, setting `by_event_type=True` returns a list of `Evoked` objects, one for each event type.

In [ ]:
# Average across events of the same type only
evoked_vis = epochs[["visual/left", "visual/right"]].average(by_event_type=True)
for evoked in evoked_vis:
    print(evoked)
    evoked.plot();

### Part 2 - Controlling how event counts are handled

You may have noticed above that the event counts for each type of event were not equal:
- `"auditory/left"` stimuli occur 72 times
- `"auditory/right"` stimuli occur 73 times
- `"visual/left"` stimuli occur 73 times
- `"visual/right"` stimuli occur 71 times

In [ ]:
# Load the sample data
raw = mne.io.read_raw_fif(
    os.path.join(mne.datasets.sample.data_path(), "MEG", "sample", "sample_audvis_raw.fif")
)
raw.pick(picks=["eeg", "stim"])
raw.del_proj()

# Generate the events array
events = mne.find_events(raw, stim_channel="STI 014")

# Choose the events to create epochs around
event_id = {
    "auditory/left": 1,
    "auditory/right": 2,
    "visual/left": 3,
    "visual/right": 4,
}

# Create the epochs
epochs = mne.Epochs(raw=raw, events=events, event_id=event_id, tmin=-0.25, tmax=0.75)

#### Equalising event counts

You may wish to use an equal number of events for each type when creating ERPs, e.g. for statistical purposes.

This is easily done using the [`equalize_event_counts()`](https://mne.tools/stable/generated/mne.Epochs.html#mne.Epochs.equalize_event_counts) method of the `Epochs` object (i.e. before creating the `Evoked` object).

By default, the number of events will be equalised:
- for all event types (`event_ids=None`)
- according to those events which are temporally closest to one another (`method="mintime"`)

In [ ]:
# Equalise the number of epochs in each condition
epochs.copy().equalize_event_counts()

**Exercises - Equalising event counts**

**Exercise:** Use the `event_ids` parameter of `equalize_event_counts()` to specify the auditory stimuli to have an equal number of events.

In [ ]:
## CODE GOES HERE

**Exercise:** Use the `event_ids` parameter of `equalize_event_counts()` to specify the visual stimuli to have an equal number of events.

In [ ]:
## CODE GOES HERE

**Exercise:** Use the `method` parameter of `equalize_event_counts()` to specify that event counts should be equalised by dropping the last events.

How do the IDs of the dropped epochs compare to when the default `"mintime"` is used?

In [ ]:
## CODE GOES HERE

#### Combining information from multiple `Evoked` objects

If equalised event counts are not a concern when you compute evoked data, it is important to consider what happens when you combine this data across multiple `Evoked` objects.

Data can be combined across multiple `Evoked` objects using the [`combine_evoked()`](https://mne.tools/stable/generated/mne.combine_evoked.html) function.

`combine_evoked()` takes a list of `Evoked` objects, as well as a way to weight the information from these `Evoked` objects (the `weights` parameter).

The weighting approaches are: `"nave"`; `"equal"`; or a list of floats.

##### `"nave"` weighting

With `"nave"`, the data from each `Evoked` object is weighted proportionally to the number of epochs that `Evoked` object was averaged across.

For example, if you have one `Evoked` object from 5 auditory events and another from 15 visual events:
- The evoked auditory data will be weighted by $\frac{1}{4}$.
- The evoked visual data will be weighted by $\frac{3}{4}$.

This may help to reduce noise in your evoked data, however it biases the final evoked data towards those events with a greater number, e.g. it could make it appear as if the brain's response to visual stimuli is much stronger than to auditory stimuli!

An example of `"nave"` weighting is shown below.

In [ ]:
# Create ERPs for two event types
evoked = epochs[["visual/right", "auditory/right"]].average(by_event_type=True)

# Combine ERPs by weighting according to number of events per type
evoked_nave = mne.combine_evoked(all_evoked=evoked, weights="nave")
evoked_nave.plot()
evoked_nave

Notice how the weightings differ slightly for each condition.

This reflects the fact that there were 73 auditory stimuli and 71 visual stimuli.

Dividing these counts by the total number of events (144) gives the 0.507 and 0.493 weightings for auditory and visual events, respectively.

##### `"equal"` weighting

The alternative approach is to weight the epochs of each event type equally.

Using the same example of 5 auditory events and 15 visual events:
- The evoked auditory data will be weighted by $\frac{1}{2}$.
- The evoked visual data will be weighted by $\frac{1}{2}$.

This avoids biases arising from differences in the number of events, but may lead to noiser results.

**Exercise:** Combine the visual and auditory ERPs with an `"equal"` weighting.

How do the results and reported weightings compare to the approach above?

In [ ]:
## CODE GOES HERE

##### Custom weightings

Finally, custom weightings for the `Evoked` objects can also be supplied.

This is done as a list of floats, with one for each `Evoked` object.

**Exercise:** Provide a custom weighting for the stimuli, weighting the visual stimuli by 0.9 and the auditory stimuli by 0.1.

How do the results compare to the weighting approaches above?

In [ ]:
## CODE GOES HERE

##### Summary of different weighting approaches

Since the number of left and right visual events were so similar, weighting according to the number of events per type (`"nave"`) or providing equal weights (`"equal"`) gives very similar results.

Naturally, our custom weighting skewed the evoked responses heavily towards the auditory stimuli.

However, this custom weighting also mimics scenarios where there is a large difference in the number of events per condition (e.g. 90 auditory stimulus trials and 10 visual stimulus trials), where weighting according to the total number of events may have a large effect on your interpretation of the data.

### Part 3 - Creating `Evoked` objects from arrays

Like for `Raw` and `Epochs` objects, `Evoked` objects can also be created from data arrays, using the [`EvokedArray`](https://mne.tools/stable/generated/mne.EvokedArray.html) class.

Below, we generate some signals as sine waves, reshape them into continuous epochs, and then average across to create 'evoked' data.

In [ ]:
# Simulation settings
duration = 10  # seconds
sfreq = 200  # sampling rate (Hz)
epoch_duration = 2  # seconds
n_epochs = duration // epoch_duration
np.random.seed(44)  # for reproducibility

# Timepoints of the simulated data
times = np.linspace(start=0, stop=duration, num=sfreq * duration, endpoint=False)

# Generate timeseries signals
data_raw = np.array(
    [
        np.sin(2 * np.pi * times * 1),  # 1 Hz sine wave
        np.sin(2 * np.pi * times * 3),  # 3 Hz sine wave
    ]
)
n_channels = data_raw.shape[0]
print(f"Shape of timeseries data: {data_raw.shape} (channels x times)")

# Reshape into epochs
data_epochs = np.reshape(data_raw, (n_channels, n_epochs, epoch_duration * sfreq))
data_epochs = np.transpose(data_epochs, (1, 0, 2))
print(f"Shape of epoched data: {data_epochs.shape} (epochs x channels x times)")

# Average across epochs
data_evoked = np.mean(data_epochs, axis=0)
print(f"Shape of evoked data: {data_evoked.shape} (channels x times)")

**Exercises - Creating `Evoked` objects from arrays**

**Exercise:** Using the information above, create an `Info` object for the simulated data, specifying them to be EEG channels and using the sampling frequency given above.

*Hint:* use the [`create_info()`](https://mne.tools/stable/generated/mne.create_info.html) function.

In [ ]:
## CODE GOES HERE

**Exercise:** Use the `data_evoked` and `Info` object generated above to create an `EvokedArray` object and display its properties.

In [ ]:
## CODE GOES HERE

**Exercise:** Plot the data to verify that it matches our expectations, i.e.:
- 2 channels of a 1 and 3 Hz sine wave.
- Duration of 2 seconds.

In [ ]:
## CODE GOES HERE

You should see the evoked data span from 0 to 2 seconds.

The times of the data in the `EvokedArray` object can be controlled with the `tmin` parameter.

Below, we explicitly set `tmin=0` (the default behaviour).

In [ ]:
# Create evoked data from the array with explicit tmin
info = mne.create_info(ch_names=n_channels, sfreq=sfreq, ch_types="eeg")
evoked = mne.EvokedArray(data=data_evoked, info=info, tmin=0)
evoked.plot()
evoked

**Exercise:** Create an `EvokedArray` object where the data spans the period [-1, 1] seconds.

In [ ]:
## CODE GOES HERE

**Exercise:** Create an `EvokedArray` object where the data spans the period [-0.5, 1.5] seconds.

In [ ]:
## CODE GOES HERE

Baselining of the evoked data can be controlled when creating an `EvokedArray` object using the `baseline` parameter.

Below, we explicitly set `baseline=None` (the default behaviour, i.e. no baselining).

In [ ]:
# Create evoked data from the array with explicit (lack of) baselining
info = mne.create_info(ch_names=n_channels, sfreq=sfreq, ch_types="eeg")
evoked = mne.EvokedArray(data=data_evoked.copy(), info=info, baseline=None)
evoked.plot()
evoked

**Exercise:** Create an `EvokedArray` object baselined for the first 100 ms of data.

Make sure to pass in a copy of the `data_evoked` array, like above.

*Hint:* Specifying baselines for evoked data takes the same form as for epoched data, i.e. `baseline=(start, end)`.

In [ ]:
## CODE GOES HERE

**Exercise:** Create an `EvokedArray` object baselined for the first 500 ms of data, where the data spans the period [-0.5, 1.5] seconds.

In [ ]:
## CODE GOES HERE

**Exercise:** Create an `EvokedArray` object baselined for the first 200 ms of data, where the data spans the period [-1, 1] seconds.

In [ ]:
## CODE GOES HERE

Controlling the times and baselining of evoked data are some of the most useful features when creating `EvokedArray` objects, however additional options exist for:
- Providing a label for the evoked data - `comment` parameter (default `""`)
- Specifying the number of epochs which have been averaged across - `nave` parameter (default `1`)

In [ ]:
# Create evoked data from the array with explicit comment and nave
info = mne.create_info(ch_names=n_channels, sfreq=sfreq, ch_types="eeg")
evoked = mne.EvokedArray(data=data_evoked, info=info, comment="example_data", nave=n_epochs)
evoked.plot()
evoked

## Conclusion

MNE makes generating ERP data and storing it in `Evoked` objects easy, either from averaging data of `Epochs` objects, or from data arrays.

Similarly to the `Raw` and `Epochs` classes, the `Evoked` class also has useful methods for:
- picking channels - [`pick()`](https://mne.tools/stable/generated/mne.Evoked.html#mne.Evoked.pick)
- cropping activity by time - [`crop()`](https://mne.tools/stable/generated/mne.Evoked.html#mne.Evoked.crop)
- plotting topographies of activity - [`plot_topo()`](https://mne.tools/stable/generated/mne.Evoked.html#mne.Evoked.plot_topo) and [`plot_topomap()`](https://mne.tools/stable/generated/mne.Evoked.html#mne.Evoked.plot_topomap)
- computing PSDs - [`compute_psd()`](https://mne.tools/stable/generated/mne.Evoked.html#mne.Evoked.compute_psd)

## Additional resources

MNE tutorial on `Evoked` objects: https://mne.tools/stable/auto_tutorials/evoked/10_evoked_overview.html

MNE tutorial on visualising `Evoked` objects: https://mne.tools/stable/auto_tutorials/evoked/20_visualize_evoked.html

MNE tutorial on ERP analysis: https://mne.tools/stable/auto_tutorials/evoked/30_eeg_erp.html